In [6]:
import os
import cv2
import numpy as np
import networkx as nx
import torch
import torchvision.transforms as transforms
from skimage.segmentation import felzenszwalb
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.color import rgb2gray
from tqdm import tqdm

In [5]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import skimage.feature as skf

In [7]:
def GraphSegmentation(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)  # Convert to RGB for display

    # Define mask for GrabCut
    row, col, _ = image.shape
    mask = np.zeros((row, col), np.uint8)

    # Initialize models correctly
    bgdModel = np.zeros((1, 65), np.float64)  # Background model
    fgdModel = np.zeros((1, 65), np.float64)  # Foreground model

    x0, y0 = 1, 1  # Ensure valid rectangle coordinates
    x1, y1 = col , row
    rect = (x0, y0, x1 - x0, y1 - y0)

    # Run GrabCut
    cv.grabCut(image, mask, rect, bgdModel, fgdModel, 10, cv.GC_INIT_WITH_RECT)

    # Modify mask to extract foreground
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
    segmented = image * mask2[:, :, np.newaxis]
    return segmented
        


In [12]:
def ExtractFeatures(image):

    if len(image.shape) == 3:  
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    edges = cv.Canny(image, 50, 150)
    edge_count = np.sum(edges > 0)

    _, binary_mask = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    binary_mask = binary_mask // 255  # Normalize to 0 and 1
    
    crack_pixels = np.sum(binary_mask)
    total_pixels = binary_mask.size
    crack_ratio = crack_pixels / total_pixels

    glcm = skf.graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = skf.graycoprops(glcm, 'contrast')[0, 0]
    correlation = skf.graycoprops(glcm, 'correlation')[0, 0]
    energy = skf.graycoprops(glcm, 'energy')[0, 0]
    homogeneity = skf.graycoprops(glcm, 'homogeneity')[0, 0]
    return [crack_ratio, edge_count, contrast, correlation, energy, homogeneity]




In [30]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bar

# Define the dataset path and categories
dataset_path = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\Dataset"
categories = ["Positive", "Negative"]
max_images_per_category = 5000


def process_images():
    data = []
    labels = []

    for category in categories:
        label = 1 if category.lower() == "positive" else 0
        folder_path = os.path.join(dataset_path, category)
        img_list = os.listdir(folder_path)

        # Use tqdm to show progress
        for img_name in tqdm(img_list[:max_images_per_category], desc=f"Processing {category} images"):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                continue

            segmented = GraphSegmentation(img)
            features = ExtractFeatures(segmented)

            data.append(features)
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

X, y = process_images()



Processing Positive images:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing Negative images: 100%|██████████| 5000/5000 [1:03:11<00:00,  1.32it/s]


In [33]:

feature_names = ['Crack Ratio', 'Edge Count', 'Contrast', 'Correlation', 'Energy', 'Homogeneity']
df = pd.DataFrame(X, columns=feature_names)
df['Label'] = y


df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to Excel
output_path = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\processed_data.xlsx"
df.to_excel(output_path, index=False)

print(f"Shuffled data saved to: {output_path}")

Shuffled data saved to: C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\processed_data.xlsx


In [1]:
import pandas as pd
file_path = r"C:\Users\EDWIN\OneDrive\Documents\GitHub\Surface-Crack-Detection-using-Graphs\processed_data.xlsx"

# Load the Excel file
df = pd.read_excel(file_path, engine="openpyxl")  # Use engine='openpyxl' if needed

# Save as CSV
df.to_csv("dataset.csv", index=False)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
file_path = r"C:\Users\EDWIN\OneDrive\Documents\GitHub\Surface-Crack-Detection-using-Graphs\dataset.csv"
df = pd.read_csv(file_path)

# Separate features and labels
X = df.drop(columns=["Label"]).values  # Features
y = df["Label"].values  # Labels

In [8]:
df.head()


,Crack Ratio,Edge Count,Contrast,Correlation,Energy,Homogeneity,Label
0,0.005900,126,36.640443,0.857151,0.993412,0.994367,0
1,0.100138,1150,76.289306,0.925161,0.896154,0.920410,1
2,0.037843,764,168.343281,0.883286,0.959125,0.963212,1
3,0.021658,621,120.072629,0.712150,0.973452,0.974232,1
4,0.075686,1066,124.648357,0.851828,0.916244,0.929077,1


In [9]:
df.tail()

,Crack Ratio,Edge Count,Contrast,Correlation,Energy,Homogeneity,Label
9995,0.004696,153,55.105181,0.811642,0.994445,0.995541,0
9996,0.009742,207,103.766988,0.888381,0.988987,0.991011,0
9997,0.008248,140,64.701337,0.912403,0.990936,0.993455,0
9998,0.068175,879,75.580640,0.864626,0.928057,0.940114,1
9999,0.000000,0,0.000000,1.000000,1.000000,1.000000,0


In [10]:
# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [11]:
# Define the model
model = Sequential([
    Dense(64, activation="relu", input_shape=(X_train.shape[1],)),  
    Dropout(0.3),  # Dropout for regularization
    Dense(32, activation="relu"),
    Dense(1, activation="sigmoid")  # Sigmoid for binary classification
])

# Compile the model
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Print model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                448       
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2561 (10.00 KB)
Trainable params: 2561 (10.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [12]:
# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)


Epoch 1/50
200/200 [==============================] - 2s 5ms/step - loss: 0.3658 - accuracy: 0.8672 - val_loss: 0.2029 - val_accuracy: 0.9212
Epoch 2/50
200/200 [==============================] - 1s 3ms/step - loss: 0.2170 - accuracy: 0.9270 - val_loss: 0.1783 - val_accuracy: 0.9362
Epoch 3/50
200/200 [==============================] - 1s 3ms/step - loss: 0.1991 - accuracy: 0.9348 - val_loss: 0.1859 - val_accuracy: 0.9312
Epoch 4/50
200/200 [==============================] - 1s 3ms/step - loss: 0.1901 - accuracy: 0.9383 - val_loss: 0.1684 - val_accuracy: 0.9450
Epoch 5/50
200/200 [==============================] - 1s 3ms/step - loss: 0.1913 - accuracy: 0.9394 - val_loss: 0.1721 - val_accuracy: 0.9419
Epoch 6/50
200/200 [==============================] - 1s 3ms/step - loss: 0.1852 - accuracy: 0.9408 - val_loss: 0.1653 - val_accuracy: 0.9444
Epoch 7/50
200/200 [==============================] - 1s 3ms/step - loss: 0.1846 - accuracy: 0.9414 - val_loss: 0.1654 - val_accuracy: 0.9431
Epoch 

In [13]:
# Evaluate on test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")


63/63 [==============================] - 0s 2ms/step - loss: 0.1651 - accuracy: 0.9415
Test Accuracy: 0.9415


In [14]:
model.save("crack_detection_model.h5")

c:\Users\EDWIN\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
